# Digits image recognition

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_csv("train.csv")
df.shape

(42000, 785)

In [3]:
y = df["label"]
X = df.drop(["label"], axis=1)

In [8]:
X = np.array(X).reshape((-1, 28, 28, 1)) / 255

In [9]:
gen = ImageDataGenerator(validation_split = 0.2)

In [10]:
def Charts(history):
    plt.plot(history.history['loss']) # Loss function value on train
    plt.plot(history.history['val_loss']) # Loss on validation dataset
    plt.title('model loss') 
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['accuracy']) # Accuracy on train
    plt.plot(history.history['val_accuracy']) # Accuracy on validation
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [12]:
from pickletools import optimize


model = Sequential()
model.add(Conv2D(32, 3, padding="same", kernel_initializer="he_normal", activation="relu", input_shape=(28, 28, 1)))
model.add(MaxPool2D(2))
model.add(BatchNormalization())

model.add(Conv2D(64, 3, padding="same", kernel_initializer="he_normal", activation="relu"))
model.add(MaxPool2D(2))
model.add(BatchNormalization())

model.add(Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
model.add(MaxPool2D(2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.3))

model.add(Dense(128, kernel_initializer="he_normal", activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

model.summary()
model.compile(optimizer=Adam(learning_rate=3e-1), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                        

In [11]:
class CustomCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        acc = logs['accuracy']
        val_acc = logs['val_accuracy']
        if acc >= 0.999 and val_acc >= 0.99:
            self.model.stop_training = True

In [13]:
history = model.fit(gen.flow(X, y = y, subset = 'training', batch_size = 1600), validation_data=gen.flow(X, y = y, subset = 'validation', batch_size = 1600),
                    epochs = 200, verbose = 1, callbacks = [CustomCallbacks()])
Charts(history)

Epoch 1/200
21/21 [==============================] - 33s 2s/step - loss: 42.2438 - accuracy: 0.1090 - val_loss: 7796.6465 - val_accuracy: 0.1290
Epoch 2/200
21/21 [==============================] - 36s 2s/step - loss: 2.3331 - accuracy: 0.1062 - val_loss: 1299.5470 - val_accuracy: 0.0977
Epoch 3/200
21/21 [==============================] - 38s 2s/step - loss: 2.3409 - accuracy: 0.1049 - val_loss: 309.7121 - val_accuracy: 0.1399
Epoch 4/200
21/21 [==============================] - 25s 1s/step - loss: 3.0640 - accuracy: 0.1111 - val_loss: 26048.7441 - val_accuracy: 0.1010
Epoch 5/200
21/21 [==============================] - 38s 2s/step - loss: 2.3019 - accuracy: 0.1102 - val_loss: 13016.3799 - val_accuracy: 0.1010
Epoch 6/200
21/21 [==============================] - 40s 2s/step - loss: 2.3019 - accuracy: 0.1098 - val_loss: 3498.0640 - val_accuracy: 0.1157
Epoch 7/200
21/21 [==============================] - 38s 2s/step - loss: 2.3020 - accuracy: 0.1083 - val_loss: 934.7879 - val_accuracy